This notebook is written to extracted several features from the text and then the extracted features are used to classify them into the given classes. The implementation is performed using the spam dataset. The participants have to use different datasets given in the Dataset folder and perform the classification. 

You have to specify your own path where the code and datasets are present

In [ ]:
import pandas as pd 
data =  pd.read_csv('SMSSpamCollection.csv', sep = '\t', names = ['label', 'message'])
data.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
text = data['message']
class_label = data['label']

In [ ]:
import numpy as np
classes_list = ["ham","spam"]
label_index = class_label.apply(classes_list.index)
label = np.asarray(label_index)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text, label, test_size=0.33, random_state=42)

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-08-26 05:38:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-26 05:38:45--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-26 05:38:46--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip


Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
glove_file = datapath('/content/glove.6B.300d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.300d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)


/usr/local/lib/python2.7/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(400000, 300)

In [ ]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)


In [ ]:
def get_w2v_general(tweet, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += vectors[word].reshape((1, size)) #* tfidf[word]
            count += 1.
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count
        return vec
    elif aggregation == 'sum':
        return vec

In [ ]:
from sklearn.preprocessing import scale

train_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 300, model,'mean') for z in X_train]))
test_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 300, model,'mean') for z in X_test]))


In [ ]:
train_vecs_glove_mean

array([[ 1.41042086, -3.26646034, -2.12185399, ..., -1.64116441,
         1.29226491,  2.17000347],
       [-0.23354711,  0.61275259, -0.11670591, ..., -0.72044358,
        -0.23813586,  0.14290177],
       [-0.80542625,  0.44193173,  1.31955426, ...,  2.35133943,
         3.46140278, -1.13922978],
       ...,
       [ 0.71016532, -0.81178571,  0.91006119, ...,  3.18449576,
         0.77984958,  0.01273765],
       [ 0.54069084,  0.80937071,  0.14605718, ...,  1.11794501,
        -1.0467528 ,  1.95796052],
       [-0.66271877,  1.68782755,  0.02669902, ...,  0.08732145,
        -0.84801561,  1.59638037]])

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import NearestCentroid
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression




names = ["Logistic Regression", "KNN","Linear SVC","DC","SGD","RF","LinearSVC with L1-based feature selection", 
         "Bernoulli NB", "Ridge Classifier", "AdaBoost", "Perceptron","Passive-Aggresive", "Nearest C4entroid"]
classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(),
    LinearSVC(),
    DecisionTreeClassifier(),
    SGDClassifier(),
    RandomForestClassifier(),
    Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False))),
  ('classification', LinearSVC(penalty="l2"))]),
    BernoulliNB(),
    RidgeClassifier(),
    AdaBoostClassifier(),
    Perceptron(),
    PassiveAggressiveClassifier(),
    NearestCentroid()
    ]
zipped_clf = zip(names,classifiers)

In [ ]:
from time import time
from sklearn import metrics


def accuracy_summary(pipeline, x_train, y_train, x_test, y_test):
    t0 = time()
    sentiment_fit = pipeline.fit(x_train, y_train)
    y_pred = sentiment_fit.predict(x_test)
    train_test_time = time() - t0
    #accuracy = accuracy_score(y_test, y_pred)
    accuracy= (metrics.accuracy_score(y_test, y_pred))
    #print ("accuracy score: {0:.2f}%".format(accuracy*100))
    print ("train and test time: {0:.2f}s".format(train_test_time))
    print ("-"*80)
    
  
    p= metrics.precision_score(y_test, y_pred,average= 'weighted')
    r = metrics.recall_score(y_test, y_pred,average= 'weighted')
    f=metrics.f1_score(y_test, y_pred, average= 'weighted')
    
    
    return accuracy, p,r,f,train_test_time


In [ ]:
def classifier_comparator():
    result = []
    classifier=zipped_clf
    
    for n,c in classifier:
        checker_pipeline = Pipeline([
                     ('classifier', c)
        ])
        print ("Validation result for {}".format(n))
        print (c)
        #clf_accuracy,p,r,f,tt_time = accuracy_summary(checker_pipeline, train_vecs_dbow, y_train, validation_vecs_dbow, y_valid)
        #clf_accuracy,p,r,f,tt_time = accuracy_summary(checker_pipeline, train_vecs_dmc, y_train, validation_vecs_dmc, y_valid)
        clf_accuracy,p,r,f,tt_time = accuracy_summary(checker_pipeline, train_vecs_glove_mean, y_train,test_vecs_glove_mean,y_test)

        result.append((n,clf_accuracy,p,r,f,tt_time))
    return result

trigram_result = classifier_comparator()
from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.DataFrame(trigram_result)
df.columns=['classifier','acc','p','r','f1','time']

 
writer = ExcelWriter('FB_hindi_test_fasttext_sum.xlsx',engine='openpyxl')
df.to_excel(writer,'Sheet1',index=False)
writer.save()


Validation result for Logistic Regression
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


train and test time: 1.15s
--------------------------------------------------------------------------------
Validation result for KNN
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
train and test time: 4.73s
--------------------------------------------------------------------------------
Validation result for Linear SVC
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


train and test time: 1.62s
--------------------------------------------------------------------------------
Validation result for DC
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
train and test time: 1.24s
--------------------------------------------------------------------------------
Validation result for SGD
SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, 

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train and test time: 0.43s
--------------------------------------------------------------------------------
Validation result for LinearSVC with L1-based feature selection
Pipeline(memory=None,
     steps=[('feature_selection', SelectFromModel(estimator=LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
     verbose=0),
        max_features=None, n...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])
train and test time: 8.24s
--------------------------------------------------------------------------------
Validation result for Bernoulli NB
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
train and test time: 0.05s
--------------------------------------------------------------------------------
Validation result for Ridge Classifier
RidgeClassifier(alpha=1.0, class_we

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [ ]:
df.head()

,classifier,acc,p,r,f1,time
0,Logistic Regression,0.964111,0.964986,0.964111,0.964470,1.139833
1,KNN,0.963567,0.962799,0.963567,0.963012,5.130893
2,Linear SVC,0.946710,0.953405,0.946710,0.948815,0.531015
3,DC,0.924959,0.926886,0.924959,0.925832,1.072232
4,SGD,0.954867,0.953537,0.954867,0.953834,0.023961


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

from sklearn.svm import SVC
model_SVM = SVC()
model_SVM.fit(x_train, y_train)
y_pred_SVM = model_SVM.predict(x_test)
print("SVM")
print("Accuracy score =", accuracy_score(y_test, y_pred_SVM))
print(metrics.classification_report(y_test, y_pred_SVM))

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,max_depth=None,min_samples_split=2, random_state=0)
rf.fit(x_train,y_train)
y_pred_rf = rf.predict(x_test)
print("random")
print("Accuracy score =", accuracy_score(y_test, y_pred_rf))
print(metrics.classification_report(y_test, y_pred_rf))

from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(x_train,y_train)
y_pred_LR = LR.predict(x_test)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(y_test, y_pred_LR))
print(metrics.classification_report(y_test, y_pred_LR ))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(x_train,y_train)
y_pred_KNN = neigh.predict(x_test)
print("KNN")
print("Accuracy score =", accuracy_score(y_test, y_pred_KNN))
print(metrics.classification_report(y_test, y_pred_KNN ))

from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(x_train.toarray(),y_train)
y_pred_naive = naive.predict(x_test.toarray())
print("Naive Bayes")
print("Accuracy score =", accuracy_score(y_test, y_pred_naive))
print(metrics.classification_report(y_test, y_pred_naive ))

from sklearn.ensemble import GradientBoostingClassifier
gradient = GradientBoostingClassifier(n_estimators=100,max_depth=None,min_samples_split=2, random_state=0)
gradient.fit(x_train,y_train)
y_pred_gradient = gradient.predict(x_test)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(y_test, y_pred_gradient))
print(metrics.classification_report(y_test, y_pred_gradient ))

    
from sklearn.tree import DecisionTreeClassifier
decision = DecisionTreeClassifier()
decision.fit(x_train,y_train)
y_pred_decision = decision.predict(x_test)
print("Decision Tree")
print("Accuracy score =", accuracy_score(y_test, y_pred_decision))
print(metrics.classification_report(y_test, y_pred_decision ))
    

SVM
('Accuracy score =', 0.866231647634584)
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1593
           1       0.00      0.00      0.00       246

   micro avg       0.87      0.87      0.87      1839
   macro avg       0.43      0.50      0.46      1839
weighted avg       0.75      0.87      0.80      1839

random
('Accuracy score =', 0.9804241435562806)
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1593
           1       1.00      0.85      0.92       246

   micro avg       0.98      0.98      0.98      1839
   macro avg       0.99      0.93      0.95      1839
weighted avg       0.98      0.98      0.98      1839

Logistic Regression
('Accuracy score =', 0.9717237629146275)
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1593
           1       0.99      0.80      0.88       246

   micro avg       0.97      